# Extracting spike trains from nex to csv

In [ ]:
import pandas as pd

from scipy import stats
from scipy.stats import mannwhitneyu as mw
from scipy.stats import sem
from scipy.stats import iqr
from scipy.stats import ttest_ind
from scipy.stats import ttest_rel
from scipy.stats import wilcoxon

import random

import math

import EntropyHub as EH

import numpy as np

import time

from os import listdir
from os.path import isfile, join

import neo

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Function for extracting only background activity


def several_spt_and_fons(fon_times, fon_durations, spt_times):
    new_spt = []
    new_isi_list = []
    prev_fon_time = 0
    prev_fon_duration = 0
    prev_fon_duration_list = []

    for fon_time, fon_duration in zip(fon_times, fon_durations):
        # print('fon begins:',fon_time)
        # print('fon end:',fon_time+fon_duration)
        int_time = float(fon_time) - sum(
            prev_fon_duration_list
        )  # interval for shifting spikes to the left
        for spt_time in spt_times:

            if spt_time >= fon_time and spt_time <= fon_time + fon_duration:

                new_spt.append(
                    float(spt_time) - int_time
                )  # add a left shifted spike to the result

        # save the previous background for processing in the next cycle
        prev_fon_time = float(fon_time)
        prev_fon_duration = float(fon_duration)
        prev_fon_duration_list.append(prev_fon_duration)

    new_spt = np.sort(drop_sim_spikes(new_spt))
    new_isi_list = [(new_spt[i] - new_spt[i - 1]) for i in range(1, len(new_spt))]

    return new_spt, new_isi_list


def drop_sim_spikes(spt_times):
    return list(set(spt_times))

In [ ]:


df_dict = {
    "patient": [],
    "doc_name": [],
    "data_name": [],
    "interval_name": [],
    "spt_list": [],
    "isi_list": [],
}

pat_path = r"./path/to/folder/with/nex/files"

ISI_total = []
ISI_change_total = []
ISI_rel_change_total = []


spike_c = 0

# patients ID list
pat_list = listdir(pat_path)

counter = 0
c = 0
# patients processing
for patient in pat_list:
    print("Patient processing:", patient)

    temp_c = 0

    start = time.time()
    folder_path = pat_path + patient
    onlyfiles = [f for f in listdir(folder_path) if isfile(join(folder_path, f))]
    # print(onlyfiles)
    c += len(onlyfiles)
    for f_name in onlyfiles:
        if f_name[-4:] == ".nex":
            nex_reader = neo.io.NeuroExplorerIO(
                filename=folder_path + "/" + str(f_name)
            )
            data = nex_reader.read()
            num_of_spt = len(data[0].segments[0].spiketrains)
            num_of_fons = len(data[0].segments[0].epochs)
            spt_names = []
            for el in data[0].segments[0].spiketrains:
                spt_names.append(el.name)
            fon_names = []
            for el in data[0].segments[0].epochs:
                fon_names.append(el.name)

            # fon = "AllFile" AND ONLY!

            if (
                num_of_fons == 1
                and str(data[0].segments[0].epochs[0].name) == "AllFile"
            ):
                fon_to_work = data[0].segments[0].epochs[0]
                for spt in data[0].segments[0].spiketrains:
                    spt_times, isi_list = several_spt_and_fons(
                        list(el.times), list(el.durations), list(spt.times)
                    )
                    df_dict["patient"].append(patient)
                    df_dict["doc_name"].append(f_name)
                    df_dict["data_name"].append(spt.name)
                    df_dict["interval_name"].append(fon_to_work.name)
                    df_dict["spt_list"].append(spt_times)
                    df_dict["isi_list"].append(isi_list)

                    spike_c += 1
                    temp_c += 1

            # multiple backgrounds in file

            if num_of_fons > 1:

                # fon = "fon" AND ONLY THAT!

                if "fon" in fon_names:
                    # looking for which of the backgrounds has the name "fon"
                    for fon_ind in range(len(data[0].segments[0].epochs)):
                        if data[0].segments[0].epochs[fon_ind].name == "fon":
                            fon_to_work = (
                                data[0].segments[0].epochs[fon_ind]
                            )  # selected background with name "fon"
                            for spt in (
                                data[0].segments[0].spiketrains
                            ):  # put all spt on fon
                                spt_times, isi_list = several_spt_and_fons(
                                    list(fon_to_work.times),
                                    list(fon_to_work.durations),
                                    list(spt.times),
                                )
                                df_dict["patient"].append(patient)
                                df_dict["doc_name"].append(f_name)
                                df_dict["data_name"].append(spt.name)
                                df_dict["interval_name"].append(fon_to_work.name)
                                df_dict["spt_list"].append(spt_times)
                                df_dict["isi_list"].append(isi_list)

                                spike_c += 1
                                temp_c += 1

                # fon = "AllFile" and spt.name

                elif "fon" not in fon_names:
                    for spt in data[0].segments[0].spiketrains:
                        if (
                            "fon_" + str(spt.name) in fon_names
                        ):  # check if there is a suitable fon for spt
                            for el in (
                                data[0].segments[0].epochs
                            ):  # find a suitable fon for spt
                                if str(el.name) == "fon_" + str(spt.name):
                                    fon_to_work = el
                                    spt_times, isi_list = several_spt_and_fons(
                                        list(fon_to_work.times),
                                        list(fon_to_work.durations),
                                        list(spt.times),
                                    )
                                    df_dict["patient"].append(patient)
                                    df_dict["doc_name"].append(f_name)
                                    df_dict["data_name"].append(spt.name)
                                    df_dict["interval_name"].append(fon_to_work.name)
                                    df_dict["spt_list"].append(spt_times)
                                    df_dict["isi_list"].append(isi_list)

                                    spike_c += 1
                                    temp_c += 1

                        else:  # put the remaining spt on AllFile
                            fon_to_work = data[0].segments[0].epochs[0]
                            spt_times, isi_list = several_spt_and_fons(
                                list(fon_to_work.times),
                                list(fon_to_work.durations),
                                list(spt.times),
                            )
                            df_dict["patient"].append(patient)
                            df_dict["doc_name"].append(f_name)
                            df_dict["data_name"].append(spt.name)
                            df_dict["interval_name"].append(fon_to_work.name)
                            df_dict["spt_list"].append(spt_times)
                            df_dict["isi_list"].append(isi_list)

                            spike_c += 1
                            temp_c += 1

                else:
                    print("Unaccounted files")
                    print(patient, f_name)
                    print()
    print("    ", temp_c)


print("Patient processing is over")
print("Spiketrains count", spike_c)

In [ ]:
df_spt_isi = pd.DataFrame(df_dict)
df_spt_isi.to_csv('spt_isi.csv')